In [1]:
#download the data
import keras
from keras.datasets import imdb 
import numpy as np

top_words = 5000 
INDEX_FROM = 3

# # set allow_pickle=True
np.load.__defaults__=(None, True, True, 'ASCII')

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

# # Reset allow_pickle=False
np.load.__defaults__=(None, False, True, 'ASCII')

C:\Users\EAZETNG\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\EAZETNG\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\EAZETNG\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\EAZETNG\Anaconda3\lib\site-packages\tensorflow\python\fr

In [2]:
#reverse lookup
word_to_id = keras.datasets.imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
id_to_word = {value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in X_train[0] ))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly <UNK> was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little <UNK> that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big <UNK> for the whole film but these children are amazing and should be <UNK> for what they

In [3]:
#one hot encode your documents
from numpy import array
from keras.preprocessing.text import one_hot
docs = ['Gut gemacht',
        'Gute arbeit',
        'Super idee',
        'Perfekt erledigt',
        'exzellent',
        'naja',
        'Schwache arbeit.',
        'Nicht gut',
        'Miese arbeit.',
        'Hätte es besser machen können.']
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[37, 45], [9, 34], [10, 3], [41, 19], [31], [22], [33, 34], [31, 37], [14, 34], [17, 29, 31, 17, 1]]


In [4]:
# Truncate and pad the review sequences 
from keras.preprocessing import sequence 
max_review_length = 500 
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length) 
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length) 

In [5]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Build the model 
embedding_vector_length = 32 
model = Sequential() 
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length)) 
model.add(LSTM(100)) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
#Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=64) 

C:\Users\EAZETNG\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Instructions for updating:
Use tf.cast instead.
Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 339s 14ms/step - loss: 0.4487 - acc: 0.7787 - val_loss: 0.5146 - val_acc: 0.8314
Epoch 2/3
25000/25000 [==============================] - 340s 14ms/step - loss: 0.2814 - acc: 0.8870 - val_loss: 0.3074 - val_acc: 0.8762
Epoch 3/3
25000/25000 [==============================] - 335s 13ms/step - loss: 0.2349 - acc: 0.9093 - val_loss: 0.3266 - val_acc: 0.8733


In [8]:
#Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.33%


In [ ]:
# #predict sentiment from reviews
# # bad = "this movie was terrible and bad"
# # good = "i really liked the movie and had fun"

# bad = "i hate been here, this place is terrible"
# good = "i love this moment, you're so awesome"

# for review in [good,bad]:
#     tmp = []
#     for word in review.split(" "):
#         tmp.append(word_to_id[word])
#     tmp_padded = sequence.pad_sequences([tmp], maxlen=max_review_length) 
#     print("%s. Sentiment: %s" % (review,model.predict(array([tmp_padded][0]))[0][0]))

In [14]:
word_index = keras.datasets.imdb.get_word_index()

# sample = 'I hate been here, this place is terrible'
sample = "I love this moment, you're so awesome"
sample_label = 1
# convert input sentence to tokens based on word_index
inps = [word_index[word] for word in sample.split() if word in word_index]
# the sentence length should be the same as the input sentences
inps = sequence.pad_sequences([inps], maxlen=500)

In [15]:
print('Accuracy:', model.evaluate(inps, [sample_label], batch_size=1)[1])
print('Sentiment score: {}'.format(model.predict(inps)[0][0]))

1/1 [==============================] - 0s 54ms/step
Accuracy: 0.0
Sentiment score: 0.30634719133377075
